In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
# Tells matplotlib to display images inline instead of a new window
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
random.seed(1000)

from time import time
import timeit #imports timeit module

C:\Users\LMS-Khatrib\AppData\Local\Temp\ipykernel_5540\641145125.py:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures

import sklearn.linear_model as skl_lm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import neighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeRegressor 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
df2017 = pd.read_csv('SRER_2017_training_bi.csv', na_values = '?').dropna()
df2021 = pd.read_csv("SRER21_dataset_v1.csv", na_values = '?').dropna()

df2017 = df2017.reindex(columns=["OID_", "Id", "gridcode","Shape_Length", "Shape_Area","CH_mean", "ARVI_mean","ARVI_max","ARVI_med","EVI_mean","EVI_max","EVI_med","NDVI_mean","NDVI_max","NDVI_med","SAVI_mean","SAVI_max","SAVI_med", "Veg_class"])
df2021 = df2021.reindex(columns=["OID_", "Id", "gridcode","Shape_Length", "Shape_Area","CH_mean", "ARVI_mean","ARVI_max","ARVI_med","EVI_mean","EVI_max","EVI_med","NDVI_mean","NDVI_max","NDVI_med","SAVI_mean","SAVI_max","SAVI_med", "Veg_class"])

bigtest1_df = pd.read_csv('SRER21_pred.csv', na_values='?')
bg1 = bigtest1_df.drop(columns=["Veg_class"])

dffull2021 = bg1.dropna()
dffull2021 = dffull2021.reindex(columns = ["OID_", "Id", "gridcode","Shape_Length", "Shape_Area","CH_mean", "ARVI_mean","ARVI_max","ARVI_med","EVI_mean","EVI_max","EVI_med","NDVI_mean","NDVI_max","NDVI_med","SAVI_mean","SAVI_max","SAVI_med", "Veg_class"]).fillna(0)

In [4]:
df2017.Veg_class = df2017.Veg_class.map({'non-woody':0, 'woody':1})
df2021.Veg_class = df2021.Veg_class.map({'non-woody':0, 'woody':1})

# Predicting models (SRER 2017 -> SRER 2021)

In [5]:
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model = DecisionTreeClassifier(max_depth=6)

#criterion='entropy'
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))

acc_score = [];
Truth = [];
Output = [];

start_time = 0
for train_index, test_index in crossvalidation.split(df2017): 
    
    X_train = df2017.iloc[train_index, 5:18]
    X_test = df2017.iloc[test_index, 5:18]
    Y_train = df2017.iloc[train_index, -1]
    Y_test = df2017.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))


---Run time is 6.825481 seconds ---

Accuracy of each fold: 
 [0.6682027649769585, 0.6831797235023042, 0.6612903225806451, 0.6900921658986175, 0.6758938869665513]
Avg accuracy: 0.6757317727850153
Std of accuracy : 
0.010266066275368545

[[1428  660]
 [ 747 1504]]

              precision    recall  f1-score   support

           0       0.66      0.68      0.67      2088
           1       0.70      0.67      0.68      2251

    accuracy                           0.68      4339
   macro avg       0.68      0.68      0.68      4339
weighted avg       0.68      0.68      0.68      4339

Sensitivity: 0.6681474900044425
Specificity: 0.6839080459770115
precision: 0.6565517241379311
f1_score: 0.6699507389162561


In [6]:
model = DecisionTreeClassifier(max_depth=6)
acc_score = [];
Truth = [];
Output = [];
testing = df2021.iloc[:,-1]
model.fit(df2017.iloc[:, 5:18], df2017.iloc[:,-1])
pred_values = model.predict(df2021.iloc[:, 5:18])

acc = accuracy_score(testing, pred_values)
acc_score.append(acc)
Truth.extend(testing.values.reshape(testing.shape[0])) 
Output.extend(pred_values)

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))
print(acc_score)

Sensitivity: 0.9766120218579235
Specificity: 0.11748878923766816
precision: 0.7100271002710027
f1_score: 0.20161600615621394
[0.695077149155033]


In [7]:
random.seed(1000)
bigtest_df = dffull2021
finalPredicted = model.predict(bigtest_df.iloc[:,5:18])
bigtest_df["Veg_class"] = finalPredicted

In [8]:
woodyarea = 0
nonwoodyarea = 0 
totalarea = 0
index = -1
woody = 0
nw = 0
#print(bigtest_df.iat[index,4])
area = bigtest_df["Shape_Area"]
#print(area)

for i in bigtest_df["Veg_class"]:
    index += 1
    if i == 1:
        totalarea += bigtest_df.iat[index,4]
        woodyarea += bigtest_df.iat[index,4]
        #print(woodyarea)
        woody += 1
    if i == 0:
        totalarea += bigtest_df.iat[index,4]
        nonwoodyarea += bigtest_df.iat[index,4]
        #print(nonwoodyarea)
        nw += 1

FWCdt = woodyarea / totalarea * 100

In [9]:
#Bagging Model 

crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model = RandomForestClassifier(max_features = 10, random_state = 2)
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))
#print(model)


acc_score = [];
Truth = [];
Output = [];

for train_index, test_index in crossvalidation.split(df2017): 
    
    X_train = df2017.iloc[train_index, 5:18]
    X_test = df2017.iloc[test_index, 5:18]
    Y_train = df2017.iloc[train_index, -1]
    Y_test = df2017.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))

#statallfeatures_dic['Run Time'].append(elapsed)
#statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
#statallfeatures_dic['Standard Error'].append(np.std(acc_score))
#statallfeatures_dic['Sensitivity'].append(sensitivity)
#statallfeatures_dic['Specificity'].append(specificity)
#statallfeatures_dic['Precision'].append(precision)
#statallfeatures_dic['F1_Score'].append(f1_score)

#indaccs_dic['LogReg']=acc_score

---Run time is 55.2374446 seconds ---

Accuracy of each fold: 
 [0.6670506912442397, 0.6728110599078341, 0.6670506912442397, 0.6970046082949308, 0.6597462514417531]
Avg accuracy: 0.6727326604265995
Std of accuracy : 
0.012824585495850723

[[1422  666]
 [ 754 1497]]

              precision    recall  f1-score   support

           0       0.65      0.68      0.67      2088
           1       0.69      0.67      0.68      2251

    accuracy                           0.67      4339
   macro avg       0.67      0.67      0.67      4339
weighted avg       0.67      0.67      0.67      4339

Sensitivity: 0.6650377609951132
Specificity: 0.6810344827586207
precision: 0.6534926470588235
f1_score: 0.6669793621013133


In [10]:
model = RandomForestClassifier(max_features = 10, random_state = 2)
acc_score = [];
Truth = [];
Output = [];
testing = df2021.iloc[:,-1]
model.fit(df2017.iloc[:, 5:18], df2017.iloc[:,-1])
pred_values = model.predict(df2021.iloc[:, 5:18])

acc = accuracy_score(testing, pred_values)
acc_score.append(acc)
Truth.extend(testing.values.reshape(testing.shape[0])) 
Output.extend(pred_values)

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))
print(acc_score)

Sensitivity: 0.9283060109289617
Specificity: 0.4094170403587444
precision: 0.7356970185334408
f1_score: 0.526073177758571
[0.7582659808963997]


In [11]:
random.seed(1000)
bigtest_df = dffull2021
finalPredicted = model.predict(bigtest_df.iloc[:,5:18])
bigtest_df["Veg_class"] = finalPredicted

In [12]:
woodyarea = 0
nonwoodyarea = 0 
totalarea = 0
index = -1
woody = 0
nw = 0
#print(bigtest_df.iat[index,4])
area = bigtest_df["Shape_Area"]
#print(area)

for i in bigtest_df["Veg_class"]:
    index += 1
    if i == 1:
        totalarea += bigtest_df.iat[index,4]
        woodyarea += bigtest_df.iat[index,4]
        #print(woodyarea)
        woody += 1
    if i == 0:
        totalarea += bigtest_df.iat[index,4]
        nonwoodyarea += bigtest_df.iat[index,4]
        #print(nonwoodyarea)
        nw += 1

FWCbag = woodyarea / totalarea * 100

In [13]:
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model = AdaBoostClassifier(n_estimators=500, learning_rate = 0.1, algorithm="SAMME.R", random_state=2)
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))
#print(model)


acc_score = [];
Truth = [];
Output = [];

for train_index, test_index in crossvalidation.split(df2017): 
    
    X_train = df2017.iloc[train_index, 5:18]
    X_test = df2021.iloc[test_index, 5:18]
    Y_train = df2017.iloc[train_index, -1]
    Y_test = df2021.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))

#statallfeatures_dic['Run Time'].append(elapsed)
#statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
#statallfeatures_dic['Standard Error'].append(np.std(acc_score))
#statallfeatures_dic['Sensitivity'].append(sensitivity)
#statallfeatures_dic['Specificity'].append(specificity)
#statallfeatures_dic['Precision'].append(precision)
#statallfeatures_dic['F1_Score'].append(f1_score)

#indaccs_dic['LogReg']=acc_score

---Run time is 119.5633313 seconds ---

Accuracy of each fold: 
 [0.7373271889400922, 0.706221198156682, 0.7142857142857143, 0.7016129032258065, 0.7093425605536332]
Avg accuracy: 0.7137579130323857
Std of accuracy : 
0.012486697023695197

[[ 296 1163]
 [  79 2801]]

              precision    recall  f1-score   support

           0       0.79      0.20      0.32      1459
           1       0.71      0.97      0.82      2880

    accuracy                           0.71      4339
   macro avg       0.75      0.59      0.57      4339
weighted avg       0.73      0.71      0.65      4339

Sensitivity: 0.9725694444444445
Specificity: 0.20287868403015763
precision: 0.7893333333333333
f1_score: 0.32279171210468915


In [14]:
model = AdaBoostClassifier(n_estimators=500, learning_rate = 0.1, algorithm="SAMME.R", random_state=2)
acc_score = [];
Truth = [];
Output = [];
testing = df2021.iloc[:,-1]
model.fit(df2017.iloc[:, 5:18], df2017.iloc[:,-1])
pred_values = model.predict(df2021.iloc[:, 5:18])

acc = accuracy_score(testing, pred_values)
acc_score.append(acc)
Truth.extend(testing.values.reshape(testing.shape[0])) 
Output.extend(pred_values)

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))
print(acc_score)

Sensitivity: 0.9626229508196721
Specificity: 0.21255605381165918
precision: 0.7348837209302326
f1_score: 0.3297391304347826
[0.7168258633357825]


In [15]:
random.seed(1000)
bigtest_df = dffull2021
finalPredicted = model.predict(bigtest_df.iloc[:,5:18])
bigtest_df["Veg_class"] = finalPredicted

In [16]:
woodyarea = 0
nonwoodyarea = 0 
totalarea = 0
index = -1
woody = 0
nw = 0
#print(bigtest_df.iat[index,4])
area = bigtest_df["Shape_Area"]
#print(area)

for i in bigtest_df["Veg_class"]:
    index += 1
    if i == 1:
        totalarea += bigtest_df.iat[index,4]
        woodyarea += bigtest_df.iat[index,4]
        #print(woodyarea)
        woody += 1
    if i == 0:
        totalarea += bigtest_df.iat[index,4]
        nonwoodyarea += bigtest_df.iat[index,4]
        #print(nonwoodyarea)
        nw += 1

FWCada = woodyarea / totalarea * 100

In [17]:
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model =  GradientBoostingClassifier(n_estimators = 500, 
                                           learning_rate = 0.1, 
                                           max_depth = 4, 
                                           random_state = 2)
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))

#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))
#print(model)


acc_score = [];
Truth = [];
Output = [];

for train_index, test_index in crossvalidation.split(df2017): 
    
    X_train = df2017.iloc[train_index, 5:18]
    X_test = df2017.iloc[test_index, 5:18]
    Y_train = df2017.iloc[train_index, -1]
    Y_test = df2017.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))

#statallfeatures_dic['Run Time'].append(elapsed)
#statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
#statallfeatures_dic['Standard Error'].append(np.std(acc_score))
#statallfeatures_dic['Sensitivity'].append(sensitivity)
#statallfeatures_dic['Specificity'].append(specificity)
#statallfeatures_dic['Precision'].append(precision)
#statallfeatures_dic['F1_Score'].append(f1_score)

#indaccs_dic['LogReg']=acc_score

---Run time is 307.5700605 seconds ---

Accuracy of each fold: 
 [0.6532258064516129, 0.6797235023041475, 0.6716589861751152, 0.684331797235023, 0.6597462514417531]
Avg accuracy: 0.6697372687215303
Std of accuracy : 
0.011737690865284093

[[1374  714]
 [ 719 1532]]

              precision    recall  f1-score   support

           0       0.66      0.66      0.66      2088
           1       0.68      0.68      0.68      2251

    accuracy                           0.67      4339
   macro avg       0.67      0.67      0.67      4339
weighted avg       0.67      0.67      0.67      4339

Sensitivity: 0.6805864060417592
Specificity: 0.6580459770114943
precision: 0.6564739608217869
f1_score: 0.657259028940445


In [18]:
model =  GradientBoostingClassifier(n_estimators = 500, 
                                           learning_rate = 0.1, 
                                           max_depth = 4, 
                                           random_state = 2)
acc_score = [];
Truth = [];
Output = [];
testing = df2021.iloc[:,-1]
model.fit(df2017.iloc[:, 5:18], df2017.iloc[:,-1])
pred_values = model.predict(df2021.iloc[:, 5:18])

acc = accuracy_score(testing, pred_values)
acc_score.append(acc)
Truth.extend(testing.values.reshape(testing.shape[0])) 
Output.extend(pred_values)

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))
print("Accuracy: {}".format(acc_score))


Sensitivity: 0.9191256830601093
Specificity: 0.4748878923766816
precision: 0.7410776766969909
f1_score: 0.5788466794206067
Accuracy: [0.773548861131521]


In [19]:
random.seed(1000)
bigtest_df = dffull2021
finalPredicted = model.predict(bigtest_df.iloc[:,5:18])
bigtest_df["Veg_class"] = finalPredicted

In [20]:
woodyarea = 0
nonwoodyarea = 0 
totalarea = 0
index = -1
woody = 0
nw = 0
#print(bigtest_df.iat[index,4])
area = bigtest_df["Shape_Area"]
#print(area)

for i in bigtest_df["Veg_class"]:
    index += 1
    if i == 1:
        totalarea += bigtest_df.iat[index,4]
        woodyarea += bigtest_df.iat[index,4]
        #print(woodyarea)
        woody += 1
    if i == 0:
        totalarea += bigtest_df.iat[index,4]
        nonwoodyarea += bigtest_df.iat[index,4]
        #print(nonwoodyarea)
        nw += 1

FWCgrad = woodyarea / totalarea * 100

In [21]:
print(FWCada)
print(FWCdt)
print(FWCbag)
print(FWCgrad)

77.2572034159111
81.06374687393047
64.67877324216532
60.67088099818383
